# Evolución adiabática discreta con Qiskit  
## De un estado paramagnético a uno ferromagnético en un sistema de 2 qubits

En este notebook vamos a estudiar cómo simular **evolución adiabática discreta** usando Qiskit, `QuantumCircuit` y `PauliEvolutionGate`.

La idea física:

- Sistema de **2 qubits**.
- Hamiltoniano inicial (paramagnético):
  \begin{equation}
    H_i = -X_0 - X_1
  \end{equation}
  donde $X_0 = X \otimes I$ y $X_1 = I \otimes X$.

- Hamiltoniano final (ferromagnético):
  \begin{equation}
    H_f = - Z_0 Z_1
  \end{equation}
  con $Z_0 Z_1 = Z \otimes Z$. Este favorece los estados $\lvert 00 \rangle$ y $\lvert 11 \rangle$ (alineados).

Construimos un camino adiabático:
\begin{equation}
  H_{\text{ad}}(t) = (1 - s)\, H_i + s\, H_f, 
  \quad s = \frac{t}{T} \in [0,1]
\end{equation}

En vez de un tiempo continuo, hacemos una **discretización** en $N$ pasos:

- $t_k = k \, \Delta t$ con $\Delta t = T/N$.
- En cada paso aplicamos la evolución unitaria:
  \begin{equation}
    U_k = e^{-i H_{\text{ad}}(t_k)\, \Delta t}
  \end{equation}
  usando `PauliEvolutionGate`.

Objetivo:

- Empezar en el **estado fundamental de $H_i$**, que es el estado paramagnético $\lvert + \rangle \lvert + \rangle$.
- Evolucionar hasta tiempo total $T$.
- Medir en la base computacional (base de $Z$).
- Ver cuán cerca estamos del estado ferromagnético: 
  - Estados "buenos" (sin defectos): $\lvert 00 \rangle, \lvert 11 \rangle$.
  - Estados "defectuosos": $\lvert 01 \rangle, \lvert 10 \rangle$.
- Estudiar cómo cambia la probabilidad de defectos con diferentes parámetros ($T$, número de pasos, etc.).

In [ ]:
# Si necesitas instalar Qiskit en el entorno:
# !pip install qiskit qiskit-aer

from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import PauliEvolutionGate
from qiskit_aer import AerSimulator

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Definimos los parámetros de los Hamiltonianos
g = 1.0   # fuerza del campo transversal (para Hi)
J = 1.0   # acoplamiento ferromagnético (para Hf)

# Hamiltoniano inicial: Hi = -X0 - X1
# X0 = X⊗I, X1 = I⊗X
Hi = SparsePauliOp.from_list([
    ("XI", -g),   # X en qubit 0
    ("IX", -g)    # X en qubit 1
])

# Hamiltoniano final: Hf = -Z0 Z1  (ferromagnético)
# ZZ = Z⊗Z
Hf = SparsePauliOp.from_list([
    ("ZZ", -J)
])

Hi, Hf

In [ ]:
def H_adiabatic(s: float) -> SparsePauliOp:
    """
    Devuelve el Hamiltoniano adiabático H(s) = (1-s) Hi + s Hf
    para un valor de s en [0, 1].
    """
    # SparsePauliOp soporta suma y multiplicación escalar.
    return (1 - s) * Hi + s * Hf

# Probamos para s = 0 (debería ser Hi) y s = 1 (debería ser Hf)
H0 = H_adiabatic(0.0)
H1 = H_adiabatic(1.0)

print("H(s=0):", H0)
print("H(s=1):", H1)

In [ ]:
# Parámetros de la evolución
T_total = 5.0      # tiempo total de la evolución adiabática
N_steps = 50       # número de pasos de discretización
dt = T_total / N_steps

print("Tiempo total T =", T_total)
print("Número de pasos N =", N_steps)
print("Paso de tiempo dt =", dt)

In [ ]:
def build_adiabatic_circuit(T: float, N: int) -> QuantumCircuit:
    """
    Construye un circuito de 2 qubits que realiza una evolución adiabática discreta
    desde Hi hasta Hf, en un tiempo total T usando N pasos.

    Estrategia:
    - Estado inicial: |+ +> (fundamental de Hi).
    - Por cada paso k:
        s_k = t_k / T,   t_k = k * (T/N)
        H_k = H_adiabatic(s_k)
        U_k = exp(-i H_k dt)
      implementado con PauliEvolutionGate(H_k, time=dt).
    - Luego medimos en base computacional.
    """
    dt = T / N

    qc = QuantumCircuit(2, 2)

    # Estado inicial paramagnético: |+> = (|0> + |1>)/sqrt(2)
    qc.h(0)
    qc.h(1)

    # Evolución adiabática discreta
    for k in range(N):
        t_k = (k + 0.5) * dt   # usamos el punto medio del intervalo [k*dt, (k+1)*dt]
        s_k = t_k / T
        H_k = H_adiabatic(s_k)

        evolution_gate = PauliEvolutionGate(H_k, time=dt)
        qc.append(evolution_gate, [0, 1])

    # Medimos en la base computacional (Z)
    qc.measure([0, 1], [0, 1])

    return qc

adiabatic_circuit = build_adiabatic_circuit(T_total, N_steps)
print(adiabatic_circuit)

In [ ]:
# Creamos el simulador
simulator = AerSimulator()

# Transpilamos el circuito para el backend del simulador
qc = transpile(adiabatic_circuit, simulator)

# Ejecutamos el circuito
shots = 10_000
job = simulator.run(qc, shots=shots)
result = job.result()
counts = result.get_counts()

print("Cuentas de medición:", counts)

In [ ]:
def compute_defect_probability(counts: dict) -> float:
    """
    Calcula la probabilidad de medir estados con 'defectos',
    es decir, |01> y |10>.
    """
    total = sum(counts.values())
    n_defects = counts.get("01", 0) + counts.get("10", 0)
    if total == 0:
        return 0.0
    return n_defects / total

p_defects = compute_defect_probability(counts)

print("Probabilidad de defectos (|01> y |10>):", p_defects)
print("Probabilidad de estados ferromagnéticos (|00> y |11>):", 
      (counts.get("00", 0) + counts.get("11", 0)) / sum(counts.values()))

In [ ]:
def run_adiabatic_simulation(T: float, N: int, shots: int = 10_000) -> float:
    """
    Construye el circuito adiabático para un tiempo total T y N pasos,
    lo simula y devuelve la probabilidad de defectos.
    """
    qc = build_adiabatic_circuit(T, N)
    sim = AerSimulator()
    qc_t = transpile(qc, sim)
    job = sim.run(qc_t, shots=shots)
    result = job.result()
    counts = result.get_counts()
    return compute_defect_probability(counts)

# Barrido de T manteniendo N_steps fijo
Ts = np.linspace(0.5, 8.0, 10)   # 10 valores entre 0.5 y 8.0
p_defects_list = []

for T in Ts:
    print(f"Simulando T = {T:.2f} ...")
    p_def = run_adiabatic_simulation(T, N_steps, shots=8000)
    p_defects_list.append(p_def)

# Graficamos
plt.figure()
plt.plot(Ts, p_defects_list, marker='o')
plt.xlabel("Tiempo total T")
plt.ylabel("Probabilidad de defectos (|01> + |10>)")
plt.title("Disminución de defectos con evolución más adiabática")
plt.grid(True)
plt.show()

## Discusión y posibles extensiones

Observaciones típicas para discutir:

- Para tiempos totales **pequeños**, la evolución no es adiabática:
  - El sistema no sigue el estado fundamental de $H_{\text{ad}}(t)$.
  - Aparecen muchas configuraciones "defectuosas" $\lvert 01 \rangle$ y $\lvert 10 \rangle$.

- Para tiempos totales **grandes**, el proceso se acerca más a la condición adiabática:
  - La probabilidad de terminar en estados ferromagnéticos $\lvert 00 \rangle$ y $\lvert 11 \rangle$ aumenta.
  - La probabilidad de defectos disminuye.

Puntos para profundizar (opcional):

1. **Gap de energía y criterio adiabático**  
   El tiempo mínimo requerido para ser adiabático está relacionado con el inverso del cuadrado del *gap* mínimo entre el estado fundamental y el excitado.

2. **Más qubits**  
   Extender el modelo a una cadena de más qubits con interacción ferromagnética (por ejemplo, suma de términos $Z_i Z_{i+1}$) y campo transversal $-\sum_i X_i$.

3. **Comparación con un quench súbito**  
   En lugar de una evolución suave, cambiar de golpe $H_i \rightarrow H_f$ y comparar la cantidad de defectos.

4. **Medir directamente la magnetización**  
   En este notebook miramos solo las probabilidades en la base computacional. También se puede calcular $\langle Z_0 Z_1 \rangle$ a partir de las cuentas.

Con este ejemplo, se ve:

- Cómo definir Hamiltonianos con `SparsePauliOp`.
- Cómo usar `PauliEvolutionGate` para simular evolución temporal.
- La idea de **evolución adiabática discreta**.
- Cómo conectar la física (defectos, ferromagnetismo) con resultados de simulación cuántica.